# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio
import io
import os

from PIL import Image
import requests
import sglang as sgl

from sglang.srt.conversation import chat_templates
from sglang.test.test_utils import is_in_ci
from sglang.utils import async_stream_and_merge, stream_and_merge

if is_in_ci():
    import patch


llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.38it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.10it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.03it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.39it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.27it/s]



### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Karen and I am excited to be your guide on this online journey. I have been a spiritual seeker for many years and have walked a variety of spiritual paths. My journey has been one of deepening and discovering my true nature, and I am passionate about sharing this journey with you.
In this space, we will explore the realms of the heart, mind, and spirit. We will delve into topics such as meditation, mindfulness, intuition, energy, and spiritual growth. My intention is to create a safe and nurturing environment where you can deepen your understanding of yourself and the world around you.
I believe that everyone has the potential to live a
Prompt: The president of the United States is
Generated text:  a 78-year-old man named Donald Trump. He was a real estate developer and businessman before he became president. He is known for his conservative views and his use of Twitter to communicate with the public. He has been involved in several controvers

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  Kaida. I'm a 25-year-old freelance writer and editor living in Tokyo. I enjoy reading, hiking, and trying new foods. I'm currently working on a novel and experimenting with different writing styles. I'm looking forward to meeting new people and learning about their experiences.
This self-introduction is neutral because it doesn't reveal any personal opinions or biases. It simply presents Kaida's background, interests, and goals in a straightforward and factual way. This is suitable for a professional or social setting where you want to make a good impression without revealing too much about yourself.
Here are a few tips to keep in mind when writing

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris. Paris is located in the northern part of the country. It is situated on the Seine River. The city is known for its beautiful architecture, art museums, and fashion. Paris is home to the famous Eiffel Tower, which is one of the most recognizable landmarks in the world. The city has a population of over 2.1 million people. Paris is a major cultural and economic center in Europe. It is a popular tourist destination, attracting millions of visitors each year. The city is also a hub for international business and finance. Paris is known for its romantic atmosphere and is often referred to as the City of Light

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  expected to be shaped by various factors, including technological advancements, societal needs, and ethical considerations. Here are some possible future trends in artificial intelligence:
1. Increased Adoption in Healthcare: AI is expected to play a significant role in healthcare, particularly in areas such as medical diagnosis, personalized medicine, and patient care. AI-powered systems will be able to analyze vast amounts of medical data, identify patterns, and make predictions, leading to more accurate diagnoses and better patient outcomes.
2. Rise of Explainable AI: As AI becomes more pervasive in various industries, there is a growing need for transparency and explainability in AI decision-making processes. Explainable



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  Abigail Wells. I'm a 25-year-old university student majoring in history, with a focus on the social dynamics of the 19th century. I'm originally from a small town in the Midwest, but I've been living in the city for the past three years, where I've been working as a research assistant and teaching assistant. I'm interested in exploring the intersections of culture, power, and identity. I'm a bit of a bookworm, and I enjoy reading, writing, and discussing historical and theoretical topics. I'm also quite active, and I enjoy hiking and running in my free time.
This is a

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. The city is situated in the north-central part of the country. It is one of the most famous cities in the world, known for its stunning architecture, art museums, and romantic atmosphere. Pr

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 Captain

 K

ael

in

...

 the

 data

.

 Edge

 AI

In [6]:
llm.shutdown()